# MT Metadata Example 5

## Experiment

We have covered some basics of MT metadata, lets put it together into an `Experiment`.

In [1]:
from mt_metadata.timeseries import (
    Auxiliary,
    Electric,
    Experiment,
    Magnetic,
    Run,
    Station,
    Survey,
)
from mt_metadata.timeseries.filters import CoefficientFilter

In [2]:
experiment = Experiment()

# set a common start and end time
start = "2020-01-01T00:00:00+00:00"
end = "2021-01-01T12:00:00+00:00"

kwargs = {"time_period.start": start, "time_period.end": end}
cf1 = CoefficientFilter(name="filter_01", gain=10)
cf2 = CoefficientFilter(name="filter_02", gain=100)

ch1_kwargs = {
    "time_period.start": start,
    "time_period.end": end,
    "filter.name": ["filter_01"],
}
ch2_kwargs = {
    "time_period.start": start,
    "time_period.end": end,
    "filter.name": ["filter_02"],
}

for survey in ["One", "Two"]:
    survey_obj = Survey(id=survey)
    survey_obj.acquired_by.author = "M T"
    survey_obj.project_lead.author = "T M"
    if survey in ["One"]:
        survey_obj.filters[cf1.name] = cf1
    else: 
        survey_obj.filters[cf2.name] = cf2
    for station in ["mt01", "mt02"]:
        station_obj = Station(id=station, **kwargs)
        for run in ["mt01a", "mt01b"]:
            run_obj = Run(id=run, **kwargs)
            if survey in ["One"]:
                for ch in ["ex", "ey"]:
                    ch_obj = Electric(component=ch, **ch1_kwargs)
                    run_obj.channels.append(ch_obj)
                for ch in ["hx", "hy", "hz"]:
                    ch_obj = Magnetic(component=ch, **ch1_kwargs)
                    run_obj.channels.append(ch_obj)
                for ch in ["temperature", "voltage"]:
                    ch_obj = Auxiliary(component=ch, **ch1_kwargs)
                    run_obj.channels.append(ch_obj)
            else:
                for ch in ["ex", "ey"]:
                    ch_obj = Electric(component=ch, **ch2_kwargs)
                    run_obj.channels.append(ch_obj)
                for ch in ["hx", "hy", "hz"]:
                    ch_obj = Magnetic(component=ch, **ch2_kwargs)
                    run_obj.channels.append(ch_obj)
                for ch in ["temperature", "voltage"]:
                    ch_obj = Auxiliary(component=ch, **ch2_kwargs)
                    run_obj.channels.append(ch_obj)
            run_obj.update_time_period()
            station_obj.runs.append(run_obj)
            station_obj.update_time_period()
        survey_obj.stations.append(station_obj)
        survey_obj.update_time_period()
    experiment.surveys.append(survey_obj)

In [3]:
experiment

Experiment Contents
--------------------
Number of Surveys: 2
  Survey ID: One
  Number of Stations: 2
  Number of Filters: 1
  --------------------
    Filter Name: filter_01
    Filter Type: coefficient
    --------------------
    Station ID: mt01
    Number of Runs: 2
    --------------------
      Run ID: mt01a
      Number of Channels: 7
      Recorded Channels: ex, ey, hx, hy, hz, temperature, voltage
      Start: 2020-01-01T00:00:00+00:00
      End:   2021-01-01T12:00:00+00:00
      --------------------
      Run ID: mt01b
      Number of Channels: 7
      Recorded Channels: ex, ey, hx, hy, hz, temperature, voltage
      Start: 2020-01-01T00:00:00+00:00
      End:   2021-01-01T12:00:00+00:00
      --------------------
    Station ID: mt02
    Number of Runs: 2
    --------------------
      Run ID: mt01a
      Number of Channels: 7
      Recorded Channels: ex, ey, hx, hy, hz, temperature, voltage
      Start: 2020-01-01T00:00:00+00:00
      End:   2021-01-01T12:00:00+00:00
    

In [4]:
d = experiment.to_dict()

In [5]:
ex = Experiment()
ex.from_dict(d)

In [6]:
ex

Experiment Contents
--------------------
Number of Surveys: 2
  Survey ID: One
  Number of Stations: 2
  Number of Filters: 1
  --------------------
    Filter Name: filter_01
    Filter Type: coefficient
    --------------------
    Station ID: mt01
    Number of Runs: 2
    --------------------
      Run ID: mt01a
      Number of Channels: 7
      Recorded Channels: ex, ey, hx, hy, hz, temperature, voltage
      Start: 2020-01-01T00:00:00+00:00
      End:   2021-01-01T12:00:00+00:00
      --------------------
      Run ID: mt01b
      Number of Channels: 7
      Recorded Channels: ex, ey, hx, hy, hz, temperature, voltage
      Start: 2020-01-01T00:00:00+00:00
      End:   2021-01-01T12:00:00+00:00
      --------------------
    Station ID: mt02
    Number of Runs: 2
    --------------------
      Run ID: mt01a
      Number of Channels: 7
      Recorded Channels: ex, ey, hx, hy, hz, temperature, voltage
      Start: 2020-01-01T00:00:00+00:00
      End:   2021-01-01T12:00:00+00:00
    

In [7]:
experiment == ex

True